In [1]:
import pandas as pd
import os

In [3]:
df = pd.read_csv('/Users/danielepica/Documents/GitHub/recommender_system_llm/src/books_recommendation.csv')
df = df[:100]

In [4]:
df.head()

,books_info
0,title:The Church of Christ: A Biblical Ecclesi...
1,title:No place to cry: The hurt and healing of...
2,"title:Night World: Daughters Of Darkness, desc..."
3,"title:The Rabbi's Cat, description:Gaining the..."
4,"title:From Potter's Field, description:The six..."


In [5]:
#!pip install chromadb langchain
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
import os

embeddings = OllamaEmbeddings(model='mxbai-embed-large', show_progress=True)
llm = Ollama(model='llama3')

#create a new Chroma vector store (index)
persist_directory = 'db_persist'
#Check if the directory already exists
if os.path.exists(persist_directory):
    #Remove the directory and its contents
    import shutil
    shutil.rmtree(persist_directory)

recommend_db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

# Crea embeddings riga per riga e salva in Chroma
for idx, row in df.iterrows():
    # Estrarre l'informazione combinata
    document_text = row['books_info']
   
    # Aggiungere il documento alla base di dati vettoriale Chroma
    recommend_db.add_texts(texts=[document_text], metadatas=[{'user_id': idx}])

# Persistenza dei dati
recommend_db.persist() #NON è PIù NECESSARIO


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 20.36it/s]
/opt/anaconda3/envs/LangChain/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [8]:
#1 Metodo
query = "If i liked Economics in one lesson, What other books like it could you suggest to me?"
docs = recommend_db.similarity_search(query, k=5)
docs

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.53it/s]


[Document(metadata={'user_id': 5}, page_content="title:Economics in one lesson, description:With over a million copies sold, Economics in One Lesson is an essential guide to the basics of economic theory. A fundamental influence on modern libertarianism, Hazlitt defends capitalism and the free market from economic myths that persist to this day. Considered among the leading economic thinkers of the “Austrian School,” which includes Carl Menger, Ludwig von Mises, Friedrich (F.A.) Hayek, and others, Henry Hazlitt (1894-1993), was a libertarian philosopher, an economist, and a journalist. He was the founding vice-president of the Foundation for Economic Education and an early editor of The Freeman magazine, an influential libertarian publication. Hazlitt wrote Economics in One Lesson, his seminal work, in 1946. Concise and instructive, it is also deceptively prescient and far-reaching in its efforts to dissemble economic fallacies that are so prevalent they have almost become a new orthod

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

#2 Metodo
qa = RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff", retriever = recommend_db.as_retriever(), return_source_documents=True, verbose=True)

In [ ]:
query = "I'm looking for an drama book. What could you suggest to me?"
result = qa({"query": query})
result['result']

In [6]:
type(recommend_db)

langchain_community.vectorstores.chroma.Chroma

<h1> Content based <h1>

Devo dividere tutti i libri già letti dall'utente selezionato, capire il genere e consigliarne altri simili.

In [28]:
query = "page_content=title:The Five Love Languages: The Secret to Love that Lasts, authors:['Gary Chapman'], categories:['Family & Relationships'], review/score:5.0,title:The Happiest Baby on the Block: The New Way to Calm Crying and Help Your Newborn Baby Sleep Longer, authors:['Harvey Karp'], categories:['FAMILY & RELATIONSHIPS'], review/score:5.0"
query2= "what is a book more similar to this: title:The Five Love Languages: The Secret to Love that Lasts, authors:['Gary Chapman'], categories:['Family & Relationships']"
query3 = "what is a book more similar to this: title:The Church of Christ: A Biblical Ecclesiology for Today, description:In The Church of Christ: A Biblical Ecclesiology for Today, respected biblical scholar Everett Ferguson presents a genuine biblical theology of the church. By systematically examining the New Testament's teaching on the existence, meaning, and purpose of the church, providing responsible coverage of the traditional topics in ecclesiology, and carefully grounding ecclesiology in the person and work of Christ, Ferguson unveils a comprehensive model of the church that is both biblically centered and relevant to a world on the verge of the twenty-first century., authors:['Everett Ferguson'], categories:['Religion'], ratingsCount:5.0"

query4= "title:The Five Love Languages: The Secret to Love that Lasts, description:Over 20 million copies sold! A perennial New York Times bestseller for over a decade! Falling in love is easy. Staying in love—that’s the challenge. How can you keep your relationship fresh and growing amid the demands, conflicts, and just plain boredom of everyday life? In the #1 New York Times international bestseller The 5 Love Languages, you’ll discover the secret that has transformed millions of relationships worldwide. Whether your relationship is flourishing or failing, Dr. Gary Chapman’s proven approach to showing and receiving love will help you experience deeper and richer levels of intimacy with your partner—starting today. The 5 Love Languages is as practical as it is insightful. Updated to reflect the complexities of relationships today, this new edition reveals intrinsic truths and applies relevant, actionable wisdom in ways that work. Includes the Couple's Personal Profile assessment so you can discover your love language and that of your loved one., authors:['Gary Chapman'], categories:['Family & Relationships'], ratingsCount:195.0"




In [29]:
def find_similar_book(query, vectorstore, top_k=5):
    
    similar_docs = vectorstore.max_marginal_relevance_search(query, k=10)

    return similar_docs

In [30]:
find_similar_book(query2, recommend_db)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.78it/s]


[Document(metadata={'user_id': 62}, page_content="title:Surrendered, description:A New York Times bestseller, this controversial guide to improving your marriage has transformed thousands of relationships, bringing women romance, harmony, and the intimacy they crave. Like millions of women, Laura Doyle wanted her marriage to be better. But when she tried to get her husband to be more romantic, helpful, and ambitious, he withdrew—and she was lonely and exhausted from controlling everything. Desperate to be in love with her man again, she decided to stop telling him what to do and how to do it. When Doyle surrendered control, something magical happened. The union she had always dreamed of appeared. The man who had wooed her was back. The underlying principle of The Surrendered Wife is simple: The control women wield at work and with children must be left at the front door of any marriage. Laura Doyle’s model for matrimony shows women how they can both express their needs and have them me